# Introduction
<font color = green> <strong>MUST READ:</strong></font> The secure element manifest format is designed to convey the unique information about a secure elements including unique ID (e.g. serial number), public keys, and certificates. The signed manifest file generated can be used to register the device to cloud providers.

This Notebook generates the Manifest file for the connected device, Signer certificate to sign the file entries and signs the Manifest file. On completing it generates 2 files, .json is the signed device Manifest file and .crt is the signers certificate.

Before running this Notebook,
1. Make sure CryptoAuth Trust Platform is having factory reset program

### Prerequisites:
This step of the tutorial will attempt to load all the necessary modules and their dependencies on your machine. If the modules are already installed you can safely step over the next Tutorial step.

<font color = orange> <strong>Note</strong></font> 
1. Installation time for prerequisites depends upon system and network speed.
2. Installing prerequisites for the first time takes more time and watch the kernel status for progress. Following image helps to locate the Kernel status,
<center><img src="../../assets/notebook/img/kerner_status.png" alt="**Check Kernel Status**" /></center>


3. Installing prerequisites gives the following error and it can be safely ignored. Functionality remains unaffected.
    - <font color = orange> azure-cli 2.0.76 has requirement colorama~=0.4.1, but you'll have colorama 0.3.9 which is incompatible.</font>
    - <font color = orange> azure-cli 2.0.76 has requirement pytz==2019.1, but you'll have pytz 2019.3 which is incompatible. </font>

In [ ]:
import sys, os

home_path = os.path.dirname(os.path.dirname(os.path.realpath(os.getcwd())))
module_path = os.path.join(home_path, 'assets', 'python')
if not module_path in sys.path:
    sys.path.append(module_path)

from requirements_helper import requirements_installer
obj = requirements_installer(os.path.join(home_path, 'assets', 'requirements.txt'))
from trustplatform import program_flash
programmer = program_flash()

### Setup Modules and Hardware
This step loads the required modules and helper functions to perform the resource generation sequence. It also 
1. Initializes the interface with TNGTLS hardware and establishes commmunication with TNGTLS.
2. Performs device initialization to verify the actual device attached

In [ ]:
####################################### Load Modules
from cryptoauthlib import *
from common import *
from manifest_generation_helper import *
import warnings
warnings.filterwarnings('ignore')
print('Importing modules - Successful')

####################################### Initialize interface
cfg = cfg_ateccx08a_kithid_default()
cfg.devtype = get_device_type_id('ATECC608A')
cfg.cfg.atcahid.dev_interface = ATCAKitType.ATCA_KIT_I2C_IFACE
cfg.cfg.atcahid.dev_identity = 0x6A #TNGTLS device I2C address
assert_msg = "Check switch position on CryptoAuth Trust Platform board, verify connections"
if atcab_init(cfg) != ATCA_SUCCESS:
    # Check for CryptoAuth Trust Platform connection and default factory reset image
    status = programmer.check_for_factory_program()
    assert status == True, status
    assert atcab_init(cfg) == ATCA_SUCCESS, assert_msg

####################################### Verify attached device
# Get connected device type
info = bytearray(4)
assert_msg = "Cannot read the TNGTLS device information, Verify device connections on Trust Platform"
assert atcab_info(info) == ATCA_SUCCESS, assert_msg
dev_type = get_device_type_id(get_device_name(info))
# Checking if the connected device matches with selected device
if dev_type != cfg.devtype:
    print('Device is not TNGTLS, Connect TNGTLS device')
    assert atcab_release() == ATCA_SUCCESS, 'atcab_release failed... Rerun Notebook'

print('Device initialization - Successful')

### Generate the Manifest file
This step assists to perform
1. Create Manifest signing key and Certificate
2. Create Manifest file and Sign

<font color = green> <strong>Note:</strong></font> Manifests downloaded from Microchip Direct will be signed by Microchip

<font color=red>If the next step yields an exception, the ECC608A is not correctly connected or of the wrong type.</font> 

This step can take a little bit of time as all the certificates are extracted. 

In [ ]:
logger_cert = create_log_signer()
print('---------------------------------------------')
manifest_data, manifest_name = tng_data()